In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [52]:
broadband_df = pd.read_csv("/Users/anukul/Desktop/Anukul/Learning/DataScience/Obtaining/broadband_data.csv")
coverage_df = pd.read_csv("/Users/anukul/Desktop/Anukul/Learning/DataScience/Obtaining/coverage_data.csv")
lsoa_df = pd.read_csv("/Users/anukul/Desktop/Anukul/Learning/DataScience/Obtaining/Postcode_to_LSOA.csv",encoding='ISO-8859-1')

/var/folders/jx/pym939rn0_zgvv112pfcjs8r0000gn/T/ipykernel_9036/161294174.py:3: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  lsoa_df = pd.read_csv("/Users/anukul/Desktop/Anukul/Learning/DataScience/Obtaining/Postcode_to_LSOA.csv",encoding='ISO-8859-1')


In this cell, we define a function process_data that Handles missing values by filling with 0 if more than 90% of values are missing, otherwise filling with the median value, applies standardization (Z-score normalization) to numeric columns to have a mean of 0 and a standard deviation of 1.

In [53]:
def process_data(df):
    for column in df.select_dtypes(include=[np.number]).columns:
        if df[column].isna().sum() > 0.9 * len(df):
            df[column] = df[column].fillna(0)
        else:
            median_value = df[column].median()
            df[column] = df[column].fillna(median_value)
    
    # Apply log transformation
    for column in df.select_dtypes(include=[np.number]).columns:
        df[column] = np.log(df[column] + 1)
    
    return df

In [54]:
broadband_df_processed = process_data(broadband_df)
coverage_df_processed = process_data(coverage_df)

Below we have filter the broadband and coverage datasets to include only the Cornwall postcode areas ('TR' and 'PL').We merge the filtered broadband and coverage data on the postcode. We further merge with the LSOA data and filter to keep only the rows where the county is 'Cornwall'.

In [21]:
broadband_df_processed.columns

Index(['postcode', 'postcode_space', 'postcode area',
       'Median download speed (Mbit/s)', 'Average download speed (Mbit/s)',
       'Minimum download speed (Mbit/s)', 'Maximum download speed (Mbit/s)',
       'Average download speed (Mbit/s) for lines < 10Mbit/s',
       'Average download speed (Mbit/s) for lines 10<30Mbit/s',
       'Average download speed (Mbit/s) for SFBB lines',
       'Average download speed (Mbit/s) for UFBB lines',
       'Median upload speed (Mbit/s)', 'Average upload speed (Mbit/s)',
       'Minimum upload speed (Mbit/s)', 'Maximum upload speed (Mbit/s)',
       'Average upload speed (Mbit/s) for lines < 10Mbit/s',
       'Average upload speed (Mbit/s) for lines 10<30Mbit/s',
       'Average upload speed (Mbit/s) for SFBB lines',
       'Average upload speed (Mbit/s) for UFBB lines',
       'Number of connections < 2 Mbit/s (number of lines)',
       'Number of connections 2<5 Mbit/s (number of lines)',
       'Number of connections 5<10 Mbit/s (number of

In [23]:
coverage_df_processed.columns

Index(['postcode', 'All Premises', 'All Matched Premises',
       'SFBB availability (% premises)', 'UFBB availability (% premises)',
       'FTTP availability (% premises)',
       '% of premises unable to receive 2Mbit/s',
       '% of premises unable to receive 5Mbit/s',
       '% of premises unable to receive 10Mbit/s',
       '% of premises unable to receive 30Mbit/s',
       '% of premises unable meet USO',
       '% of premises able to receive decent broadband from FWA',
       '% of premises able to receive SFBB from FWA',
       '% of premises able to receive NGA', 'pcds', 'pca'],
      dtype='object')

We filter the broadband and coverage datasets to include only the Cornwall postcode areas ('TR' and 'PL')and then we  merge the filtered broadband and coverage data on the postcode. We finally merge with the LSOA data and filter to keep only the rows where the county is 'Cornwall'.

In [55]:
cornwall_broadband_data = (
    broadband_df_processed[
        broadband_df_processed['postcode area'].isin(['TR', 'PL'])
    ]
    .merge(
        coverage_df_processed[
            coverage_df_processed['pca'].isin(['TR', 'PL'])
        ],
        left_on='postcode',  
        right_on='postcode',  
        how='inner'
    )
    .rename(columns={
        'Average download speed (Mbit/s)': 'Average_download_speed_Mbit_s',
        'Maximum download speed (Mbit/s)': 'Maximum_download_speed_Mbit_s',
        'Minimum download speed (Mbit/s)': 'Minimum_download_speed_Mbit_s',
        'Average upload speed (Mbit/s)': 'Average_upload_speed_Mbit_s',
        'Maximum upload speed (Mbit/s)': 'Maximum_upload_speed_Mbit_s',
        'Minimum upload speed (Mbit/s)': 'Minimum_upload_speed_Mbit_s',
        'Average data usage (GB)': 'Average_data_usage_GB'
    })
    .merge(
        lsoa_df[['pcds', 'lsoa11nm', 'ladnm']].rename(columns={
            'pcds': 'postcode_space',
            'lsoa11nm': 'lsoa_area',
            'ladnm': 'county'
        }),
        on='postcode_space',
        how='left'
    )
    .query("county == 'Cornwall'")
    .loc[:, [
        'postcode', 'postcode_space', 'postcode area',
        'Average_download_speed_Mbit_s', 'Maximum_download_speed_Mbit_s', 'Minimum_download_speed_Mbit_s',
        'Average_upload_speed_Mbit_s', 'Maximum_upload_speed_Mbit_s', 'Minimum_upload_speed_Mbit_s',
        'Average_data_usage_GB', 'All Premises', 'All Matched Premises', 'lsoa_area', 'county'
    ]]
)


In [56]:
cornwall_broadband_data.to_csv("/Users/anukul/Desktop/Anukul/Learning/DataScience/Cleaning/cleaned-data/cornwall-broadband-cleaned.csv", index=False)

In [58]:
cornwall_broadband_data.columns

Index(['postcode', 'postcode_space', 'postcode area',
       'Average_download_speed_Mbit_s', 'Maximum_download_speed_Mbit_s',
       'Minimum_download_speed_Mbit_s', 'Average_upload_speed_Mbit_s',
       'Maximum_upload_speed_Mbit_s', 'Minimum_upload_speed_Mbit_s',
       'Average_data_usage_GB', 'All Premises', 'All Matched Premises',
       'lsoa_area', 'county'],
      dtype='object')

Similarly, We filter the broadband and coverage datasets to include only the Bristol postcode area ('BS'), merge the filtered broadband and coverage data on the postcode.We further merge with the LSOA data and filter to keep only the rows where the county is 'Bristol, City of'.

In [59]:
bristol_broadband_data = (
    broadband_df_processed[
        broadband_df_processed['postcode area'] == 'BS'
    ]
    .merge(
        coverage_df_processed[
            coverage_df_processed['pca'] == 'BS'
        ],
        left_on='postcode',  
        right_on='postcode',  
        how='inner'
    )
    .rename(columns={
        'Average download speed (Mbit/s)': 'Average_download_speed_Mbit_s',
        'Maximum download speed (Mbit/s)': 'Maximum_download_speed_Mbit_s',
        'Minimum download speed (Mbit/s)': 'Minimum_download_speed_Mbit_s',
        'Average upload speed (Mbit/s)': 'Average_upload_speed_Mbit_s',
        'Maximum upload speed (Mbit/s)': 'Maximum_upload_speed_Mbit_s',
        'Minimum upload speed (Mbit/s)': 'Minimum_upload_speed_Mbit_s',
        'Average data usage (GB)': 'Average_data_usage_GB'
    })
    .merge(
        lsoa_df[['pcds', 'lsoa11nm', 'ladnm']].rename(columns={
            'pcds': 'postcode_space',
            'lsoa11nm': 'lsoa_area',
            'ladnm': 'county'
        }),
        on='postcode_space',
        how='left'
    )
    .query("county == 'Bristol, City of'")
    .loc[:, [
        'postcode', 'postcode_space', 'postcode area',
        'Average_download_speed_Mbit_s', 'Maximum_download_speed_Mbit_s', 'Minimum_download_speed_Mbit_s',
        'Average_upload_speed_Mbit_s', 'Maximum_upload_speed_Mbit_s', 'Minimum_upload_speed_Mbit_s',
        'Average_data_usage_GB', 'All Premises', 'All Matched Premises', 'lsoa_area', 'county'
    ]]
)

In [60]:
bristol_broadband_data.to_csv("/Users/anukul/Desktop/Anukul/Learning/DataScience/Cleaning/cleaned-data/bristol-broadband-cleaned.csv", index=False)

Finally, cleaned broadband data including coverage of both region Bristol and Cornwall

In [61]:
broadband_cleaned = pd.concat([cornwall_broadband_data, bristol_broadband_data])
broadband_cleaned.to_csv("/Users/anukul/Desktop/Anukul/Learning/DataScience/Cleaning/cleaned-data/combined-broadband-cleaned.csv", index=False)